# Consulta de SALDO:

In [ ]:
# !pip install web3
from web3 import Web3

w3 = Web3(Web3.HTTPProvider('https://mainnet.infura.io/v3/seu_id_do_projeto_infura'))

from_address = '0x654BdNdlkjjlsdsd321314sdsds354134132412s' # ethereum node fake

checksum_address_from = w3.to_checksum_address(from_address)

balance_wei = w3.eth.get_balance(from_address)

balance_ether = w3.from_wei(balance_wei, 'ether')

print("Ethereum Balance:", balance_ether, "ETH")

# Função para conversão de ETH em BRL e USD:

In [2]:
import requests
import decimal

def converteEthereum(eth_amount):
    exchange_rate_api_brl = 'https://api.coingecko.com/api/v3/simple/price?ids=ethereum&vs_currencies=brl'
    response = requests.get(exchange_rate_api_brl)
    exchange_rates = response.json()
    eth_to_brl_rate = exchange_rates['ethereum']['brl']
    brl_amount = decimal.Decimal(eth_amount) * decimal.Decimal(eth_to_brl_rate)

    exchange_rate_api_dollar = 'https://api.coingecko.com/api/v3/simple/price?ids=ethereum&vs_currencies=usd'
    response = requests.get(exchange_rate_api_dollar)
    exchange_rates = response.json()
    eth_to_usd_rate = exchange_rates['ethereum']['usd']
    usd_amount = decimal.Decimal(eth_amount) * decimal.Decimal(eth_to_usd_rate)
    
    return brl_amount, usd_amount

# Cálculo da taxa GAS da transação:

In [3]:
endereco_destinatario = '0x654BdNdlkjjlsdsd321314sdsds354134132412s' #eth binance fake

checksum_address_destinatario = w3.to_checksum_address(endereco_destinatario)


valor_sacar = 0.001

print('Valor do saque: ', valor_sacar, 'ETH')
saque_real, saque_usd = converteEthereum(valor_sacar)
print(round(saque_real,2), 'BRL')
print(round(saque_usd, 2), 'USD')

valor_transacao_wei = w3.to_wei(valor_sacar, 'ether')  # Substitua pelo valor da transação desejado

parametros_transacao = {
    'from': from_address,
    'to': checksum_address_destinatario,
    'value': valor_transacao_wei,
}

# Calcular o valor do gas
gas_estimado = w3.eth.estimate_gas(parametros_transacao)
preco_gas_atual_eth = w3.from_wei(w3.eth.gas_price, 'ether')

valor_real, valor_usd = converteEthereum(preco_gas_atual_eth)

print(f'\nGAS da transação: {gas_estimado} GAS')
print(f"Em ethereum: {round(gas_estimado * preco_gas_atual_eth, 10)} ETH.")
print(f"Em reais: {round(gas_estimado * valor_real, 2)} BRL.")
print(f"Em dólar: {round(gas_estimado * valor_usd, 2)} USD.")

Valor do saque:  0.001 ETH
8.87 BRL
1.86 USD

GAS da transação: 21000 GAS
Em ethereum: 0.0002384110 ETH.
Em reais: 2.11 BRL.
Em dólar: 0.44 USD.


# Transação de saque. CUIDADO!!

In [4]:
from getpass import getpass
PALAVRAS_SECRETAS = getpass("Digite as palavras secretas: ")

Digite as palavras secretas: ········


In [ ]:
w3.eth.account.enable_unaudited_hdwallet_features()

# for i in range(1):
#     acc = w3.eth.account.from_mnemonic(PALAVRAS_SECRETAS, account_path=f"m/44'/60'/0'/0/{i}")
#     print(f"\naddress{i + 1} = '{acc.address}'")
#     print(f"private{i + 1} = '{Web3.to_hex(acc.key)}'")

acc = w3.eth.account.from_mnemonic(PALAVRAS_SECRETAS, account_path=f"m/44'/60'/0'/0/0")

print(f"\naddress = '{acc.address}'")

In [ ]:
from web3 import Web3
import os

amount_wei = w3.to_wei(valor_sacar, 'ether')

transaction = {
    'to': checksum_address_destinatario,
    'value': amount_wei,
    'gas': 21000,  # Gas limit for a standard transaction
    'gasPrice': w3.eth.gas_price,  # Use the current gas price
    'nonce': w3.eth.get_transaction_count(from_address)
}


signed_txn = w3.eth.account.sign_transaction(transaction, private_key = Web3.to_hex(acc.key))

choice = input(f'Confirma o saque de {valor_sacar} ETH? Escolha Yes or No (Y/N): ')

if choice.lower() == 'yes' or choice.lower() == 'y':
    print("Tentando efetuar a transação...")
    
#### Send the transaction
#### CUIDADO!!!ESTA LINHA EFETUA A TRANSACAO. MANTER COMENTADA!!! 

#     tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)  

elif choice.lower() == 'no' or choice.lower() == 'n':
    print("Você escolheu cancelar a transação.")
else:
    print("Escolha inválida.")


tx_hash = tx_hash.hex()

print("Hash da transação:", tx_hash)

#recibo da transação
tx_receipt = w3.eth.get_transaction_receipt(tx_hash)

import pandas as pd

if tx_receipt is not None:
    if tx_receipt['status'] == 1:
        status = "Transação bem sucedida!"
    else:
        status = "A transação falhou!"
        
    tx_values = {
        'Transaction Hash': [tx_receipt['transactionHash'].hex()],
        'Block Hash': [tx_receipt['blockHash'].hex()],
        'Block Number': [tx_receipt['blockNumber']],
        'From': [tx_receipt['from']],
        'To': [tx_receipt['to']],
        'Gas Used': [tx_receipt['gasUsed']],
        'Status': status,
        # Add more fields as needed
    }

    df = pd.DataFrame(tx_values)
    display(df.T)
else:
    print("Recibo da transação não encontrado")

# FIM DO CODIGO